In [2]:
import plotly.express as px
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import PolynomialFeatures
from sklearn.model_selection import GridSearchCV
import pandas as pd
from tqdm import tqdm
import joblib
import datetime as dt
from sklearn.model_selection import TimeSeriesSplit
from skimpy import skim
import matplotlib.pyplot as plt
import numpy as np
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.compose import make_column_transformer
from sklearn.model_selection import train_test_split

In [3]:
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import Ridge
from sklearn.linear_model import Lasso
from sklearn.linear_model import ElasticNet
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.neighbors import KNeighborsRegressor
from sklearn.svm import SVR
from sklearn.neural_network import MLPRegressor
from sklearn.linear_model import HuberRegressor
from sklearn.linear_model import RANSACRegressor
from sklearn.gaussian_process import GaussianProcessRegressor
from sklearn.gaussian_process.kernels import DotProduct, WhiteKernel, RBF

In [4]:
import warnings
warnings.filterwarnings('ignore')

In [5]:
import sys
sys.path.append('..')
import fx

In [6]:
data = fx.pull_data(days=90)

In [7]:
data.head(5)

,ANM,Non-ANM,Total,Direction,Lead_hours,Source_time,Speed
time,,,,,,,
2022-11-25 15:00:00+00:00,15.765571,18.399217,34.164788,SSW,1,1.669381e+09,8.94080
2022-11-25 18:00:00+00:00,13.754039,18.151967,31.906006,SW,1,1.669392e+09,12.07008
2022-11-25 21:00:00+00:00,12.167921,18.977528,31.145449,WSW,1,1.669403e+09,12.96416
2022-11-26 00:00:00+00:00,9.446035,17.241328,26.687362,WSW,1,1.669414e+09,11.17600
2022-11-26 03:00:00+00:00,13.945147,18.086372,32.031519,S,1,1.669424e+09,8.04672


In [8]:
anm_pipeline = Pipeline(steps=[
    ("col_transformer", ColumnTransformer(transformers=[
        ("time", None, []),
        ("Speed", None, ["Speed"]),
        ("Direction", None, ["Direction"]),
        ], remainder="drop")),
    ("model", None)
])

anm_params = {
    'col_transformer__time' : ["drop", None, fx.TimestampTransformer()],
    'col_transformer__Speed': [None, StandardScaler(), PolynomialFeatures(), fx.EmpiricalWaveletTransform(level=5)],
    'col_transformer__Direction': ["drop", fx.WindDirectionMapper(), fx.CompassToCartesianTransformer()],
    'model': [
        LinearRegression(), 
        MLPRegressor(hidden_layer_sizes=(150, 150), activation='tanh', solver='sgd'), 
        SVR(kernel='rbf', gamma='scale', C=1.0, epsilon=0.1),
        HuberRegressor(epsilon=1.35, alpha=0.0001),
        RANSACRegressor(min_samples=0.1, max_trials=100),
        GaussianProcessRegressor(alpha=0.1, kernel=RBF()) 
    ]
}

In [9]:
non_anm_pipeline = Pipeline(steps=[
    ("col_transformer", ColumnTransformer(transformers=[
        ("Speed", None, ["Speed"]),
        ("Direction", None, ["Direction"]),
        ], remainder="drop")),
    ("model", None)
])

non_anm_params = {
    'col_transformer__Speed': [None, StandardScaler(), PolynomialFeatures(), fx.EmpiricalWaveletTransform(level=5)],
    'col_transformer__Direction': ["drop", fx.WindDirectionMapper(), fx.CompassToCartesianTransformer()],
    'model': [
        LinearRegression(), 
        MLPRegressor(hidden_layer_sizes=(150, 150), activation='tanh', solver='sgd'), 
        SVR(kernel='rbf', gamma='scale', C=1.0, epsilon=0.1),
        HuberRegressor(epsilon=1.35, alpha=0.0001),
        RANSACRegressor(min_samples=0.1, max_trials=100),
        GaussianProcessRegressor(alpha=0.1, kernel=RBF()) 
    ]
}

In [10]:
tscv = TimeSeriesSplit(n_splits=5)

In [11]:
anm_gridsearch = GridSearchCV(anm_pipeline, anm_params, cv=tscv, scoring='neg_mean_squared_error', n_jobs=-1, verbose=1)
non_anm_gridsearch = GridSearchCV(non_anm_pipeline, non_anm_params, cv=tscv, scoring='neg_mean_squared_error', n_jobs=-1, verbose=1)

In [12]:
ANM_X_train, ANM_y_train, ANM_X_test, ANM_y_test = fx.data_splitting(data, output_val="ANM")
non_ANM_X_train, non_ANM_y_train, non_ANM_X_test, non_ANM_y_test = fx.data_splitting(data, output_val="Non-ANM")
total_X_train, total_y_train, total_X_test, total_y_test = fx.data_splitting(data, output_val="Total")

In [13]:
# check that all test sets are same length
len(ANM_y_test) == len(non_ANM_y_test) == len(total_y_test)

True

In [14]:
def train_models(X_train, y_train, X_test, y_test, gridsearch):
    gridsearch.fit(X_train, y_train)
    print("Best params: ", gridsearch.best_params_)
    print("Best score: ", gridsearch.best_score_)
    print("Test score: ", gridsearch.score(X_test, y_test))
    return gridsearch

In [15]:
anm_gridsearch = train_models(ANM_X_train, ANM_y_train, ANM_X_test, ANM_y_test, anm_gridsearch)

Fitting 5 folds for each of 216 candidates, totalling 1080 fits
Best params:  {'col_transformer__Direction': CompassToCartesianTransformer(), 'col_transformer__Speed': StandardScaler(), 'col_transformer__time': 'drop', 'model': MLPRegressor(activation='tanh', hidden_layer_sizes=(150, 150), solver='sgd')}
Best score:  -11.939125132865389
Test score:  -9.284098864598416


In [16]:
non_anm_gridsearch = train_models(non_ANM_X_train, non_ANM_y_train, non_ANM_X_test, non_ANM_y_test, non_anm_gridsearch)

Fitting 5 folds for each of 72 candidates, totalling 360 fits
Best params:  {'col_transformer__Direction': CompassToCartesianTransformer(), 'col_transformer__Speed': EmpiricalWaveletTransform(), 'model': MLPRegressor(activation='tanh', hidden_layer_sizes=(150, 150), solver='sgd')}
Best score:  -14.472026630357178
Test score:  -16.941399489450493


In [17]:
def predict_and_combine(ANM_X_test, non_ANM_X_test, y_test, anm_gridsearch, non_anm_gridsearch):
    anm_pred = anm_gridsearch.predict(ANM_X_test)
    non_anm_pred = non_anm_gridsearch.predict(non_ANM_X_test)
    pred = anm_pred + non_anm_pred
    print(f"Overall test score: {fx.MSE(pred, y_test)}")
    return pred

In [18]:
combined_pred = predict_and_combine(ANM_X_test, non_ANM_X_test, total_y_test, anm_gridsearch, non_anm_gridsearch)

Overall test score: 30.55529130246546


In [19]:
def save_models(anm_gridsearch, non_anm_gridsearch):
    joblib.dump(anm_gridsearch, rf"models\anm_gridsearch_{dt.date.today()}.pkl")
    joblib.dump(non_anm_gridsearch, rf"models\non_anm_gridsearch_{dt.date.today()}.pkl")
    return

In [20]:
save_models(anm_gridsearch, non_anm_gridsearch)

In [21]:
def load_models_and_train_on_all_data(data, anm_gridsearch, non_anm_gridsearch):
    # naming schemes is not my strong suit
    anm_X_train, anm_y_train = fx.final_data_splitting(data, output_val="ANM")
    non_anm_X_train, non_anm_y_train = fx.final_data_splitting(data, output_val="Non-ANM")
    anm_gridsearch.fit(anm_X_train, anm_y_train)
    non_anm_gridsearch.fit(non_anm_X_train, non_anm_y_train)
    return anm_gridsearch, non_anm_gridsearch

In [22]:
anm_model, non_anm_model = load_models_and_train_on_all_data(data, anm_gridsearch, non_anm_gridsearch)

Fitting 5 folds for each of 216 candidates, totalling 1080 fits
Fitting 5 folds for each of 72 candidates, totalling 360 fits


In [23]:
forecast = fx.load_forecasts()

In [24]:
forecast

,Direction,Lead_hours,Source_time,Speed
time,,,,
2023-02-23 15:00:00+00:00,W,10,1677124800,15.19936
2023-02-23 15:00:00+00:00,W,100,1676800800,11.17600
2023-02-23 15:00:00+00:00,W,101,1676797200,11.17600
2023-02-23 15:00:00+00:00,W,102,1676793600,11.17600
2023-02-23 15:00:00+00:00,W,104,1676786400,12.07008
...,...,...,...,...
2023-02-27 21:00:00+00:00,NNE,114,1677117600,4.02336
2023-02-27 21:00:00+00:00,NNE,115,1677114000,4.02336
2023-02-27 21:00:00+00:00,W,118,1677103200,3.12928


In [25]:
total_X_train, total_y_train = fx.final_data_splitting(data, output_val="Total")
anm_pred = anm_model.predict(forecast)
non_anm_pred = non_anm_model.predict(forecast)

In [26]:
def create_forecast_df(forecast, anm_pred, non_anm_pred):
    future = anm_pred + non_anm_pred
    forecast["Power Generation Forecast"] = future
    forecast = forecast.resample("3H").mean()
    forecast.drop(columns=["Source_time"], inplace=True)
    return forecast

In [27]:
forecast_df = create_forecast_df(forecast, anm_pred, non_anm_pred)

In [28]:
def create_final_plotting_df(forecast_df, data):
    # this code is just for plotting the final graph
    ANM_X_train, ANM_y_train, ANM_X_test, ANM_y_test = fx.data_splitting(data, output_val="ANM")
    non_ANM_X_train, non_ANM_y_train, non_ANM_X_test, non_ANM_y_test = fx.data_splitting(data, output_val="Non-ANM")
    total_X_train, total_y_train, total_X_test, total_y_test = fx.data_splitting(data, output_val="Total")

    test_anm_pred = anm_model.predict(ANM_X_test)
    test_non_anm_pred = non_anm_model.predict(non_ANM_X_test)

    test_prediction = test_anm_pred + test_non_anm_pred
    test_data = fx.create_timestamps(test_prediction, total_X_test, total_y_test)

    # slice total_x_test data to only get data up to the forecast datapoint
    total_X_test = total_X_test.loc[:forecast_df.index[0]]

    wind_speed_data = pd.concat([forecast_df["Speed"], total_X_test["Speed"]], axis=0)

    # combine testdata and forecastdf for easy plotting
    final_df = pd.concat([test_data[["predict", "actual"]], forecast_df["Power Generation Forecast"]], axis=0)
    final_df.columns = ["Model", "Actual", "Forecast"]
    return final_df, wind_speed_data

In [29]:
final_df, wind_speed_data = create_final_plotting_df(forecast_df, data)
final_df

,Model,Actual,Forecast
2023-02-08 15:00:00+00:00,27.198068,27.150278,NaN
2023-02-08 18:00:00+00:00,24.467782,29.662412,NaN
2023-02-08 21:00:00+00:00,24.183614,27.705957,NaN
2023-02-09 00:00:00+00:00,24.183614,27.060540,NaN
2023-02-09 03:00:00+00:00,23.474089,25.723872,NaN
...,...,...,...
2023-02-27 09:00:00+00:00,NaN,NaN,5.330483
2023-02-27 12:00:00+00:00,NaN,NaN,2.620143
2023-02-27 15:00:00+00:00,NaN,NaN,2.889182
2023-02-27 18:00:00+00:00,NaN,NaN,4.658887


In [31]:
# plot final_df using plotly
fig = px.line(final_df, x=final_df.index, y=["Model", "Actual", "Forecast"], title="Power Generation Forecast (Test Data and Forecasted Future)")
# add x and y axis labels
fig.update_xaxes(title_text="Date")
fig.update_yaxes(title_text="Power Generation (MW)")
# change legend heading
fig.update_layout(legend_title_text="")
# add wind speed info to hover
customdata = wind_speed_data
fig.update_traces(hovertemplate="<b>Power Generation: %{y:.2f} MW </b><br> Wind Speed: %{customdata:.2f} m/s <br><i> %{x}</i><extra></extra>", customdata=customdata)
fig.show()
